Import Libraries

In [1]:
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

File Config

In [2]:
files = ['just_dshock_dlim.mat', 
         'just_policy_dlim.mat', 
         'dshock_policy_dlim.mat',
         'just_dshock_ylim.mat', 
         'just_policy_ylim.mat', 
         'dshock_policy_ylim.mat',
         'just_dshock_klim.mat', 
         'just_policy_klim.mat', 
         'dshock_policy_klim.mat',
         'just_dshock_h_klim.mat', 
         'just_policy_h_klim.mat', 
         'dshock_policy_h_klim.mat',
         'just_dshock_h.mat', 
         'just_policy_h.mat', 
         'dshock_policy_h.mat',
         'just_dshock_h_rho0.mat', 
         'just_policy_h_rho0.mat', 
         'dshock_policy_h_rho0.mat',
         'just_dshock_h_rho10.mat', 
         'just_policy_h_rho10.mat', 
         'dshock_policy_h_rho10.mat'] 

experiment_names = ['Fixed debt limit - Debt Relief Only', 
                    'Fixed debt limit - Policy Reform Only', 
                    'Fixed debt limit- Debt Relief & Policy Reform',
                    'GDP debt limit - Debt Relief Only', 
                    'GDP debt limit - Policy Reform Only', 
                    'GDP debt limit - Debt Relief & Policy Reform',
                    'K debt limit - Debt Relief Only', 
                    'K debt limit - Policy Reform Only', 
                    'K debt limit - Debt Relief & Policy Reform',
                    'Human capital, phi = 0, K debt limit - Debt Relief Only', 
                    'Human capital, phi = 0, K debt limit - Policy Reform Only', 
                    'Human capital, phi = 0, K debt limit - Debt Relief & Policy Reform',
                    'Human capital, phi = 0.1, K debt limit - Debt Relief Only', 
                    'Human capital, phi = 0.1, K debt limit - Policy Reform Only', 
                    'Human capital, phi = 0.1, K debt limit - Debt Relief & Policy Reform',
                    'Discount Rate Policy, Human capital, phi = 0, K debt limit - Debt Relief Only', 
                    'Human capital, phi = 0, discount rate policy, K debt limit - Policy Reform Only', 
                    'Human capital, phi = 0, discount rate policy, K debt limit - Debt Relief & Policy Reform',
                    'Human capital, phi = 0.1, discount rate policy, K debt limit - Debt Relief Only', 
                    'Human capital, phi = 0.1, discount rate policy, K debt limit - Policy Reform Only', 
                    'Human capital, phi = 0.1, discount rate policy, K debt limit - Debt Relief & Policy Reform']

Read matlab file, create Dataframe

In [3]:
c_hash = {}

for file, exp_name in zip(files, experiment_names):
    data = scipy.io.loadmat(file)
    c_array = data["C"]
    c_hash[exp_name] = c_array
    #print(c_array[1])
        

Calculation

In [4]:
rho = 0.07
gamma = 2
beta = 1/(1+rho)
periods = len(c_hash[experiment_names[0]])

v_array = rho * np.array([beta ** i for i in range(1, periods)]) #compute v array

welfares = {}
for exp in experiment_names:
  c_array_ppt = c_hash[exp][1:] #c array expressed as percentage points

  c_array_decimal = 1 + c_array_ppt/100 #convert percentage points to decimal

  c_array_gammaexp = c_array_decimal ** (1-gamma) #1-gamma exponent

  dot_product = np.dot(v_array, c_array_gammaexp) #dot product

  result = dot_product ** (1/(1-gamma))

  rounded_result = round((result.item() - 1)*100, 2)

  welfares[exp] = rounded_result

In [5]:
total_sum = np.sum(v_array)
total_sum

0.9999987587459308

Display calculations

In [6]:
welfares_df = pd.DataFrame(list(welfares.items()), columns=['model', 'welfare decimal change'])
welfares_df

,model,welfare decimal change
0,Fixed debt limit - Debt Relief Only,2.74
1,Fixed debt limit - Policy Reform Only,0.56
2,Fixed debt limit- Debt Relief & Policy Reform,3.36
3,GDP debt limit - Debt Relief Only,2.70
4,GDP debt limit - Policy Reform Only,0.57
5,GDP debt limit - Debt Relief & Policy Reform,3.34
6,K debt limit - Debt Relief Only,2.57
7,K debt limit - Policy Reform Only,0.63
8,K debt limit - Debt Relief & Policy Reform,3.26
9,"Human capital, phi = 0, K debt limit - Debt Re...",3.85


Export to csv

In [7]:
welfares_df.to_csv("welfare_changes.csv", index=False)